<a href="https://colab.research.google.com/github/ChihabEddine98/AmazonAnalysis/blob/main/AmazonAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TODO LIST
--------


- [x] Genral collect for dataset
- [ ] Complete dataset
- [ ] Make settings in an external file
- [ ] Make settings in an external file
- [ ] Make requirement installations in another file
- [ ] Clean Data & prepocessing


# Requirements Installations

In [1]:
# Run just once 
!pip install webdriver-manager
!pip install selenium
!pip install requests
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     |████████████████████████████████| 911kB 5.7MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://ppa.launchpad.net/d

In [21]:
# imports
import os
import sys
import requests as rq
import numpy as np
import pandas as pd
import re 
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.utils import ChromeType
import unicodedata
import pickle as pk
# end imports

# Part 01  : Data Gathering

In [10]:
# Settings 
reviews_div_cls = 'review'


prod_urls = {'informatique':
              [
                {
                    'prod_id': 1,
                    'prod_url':
                    'https://www.amazon.fr/dp/B00VWEK4IG/ref=redir_mobile_desktop?_encoding=UTF8&aaxitk=.H.saUSQi0IDvqb-Ff4aow&hsa_cr_id=3200936120002&pd_rd_plhdr=t&pd_rd_r=01bb0b8a-0cca-4f13-b9af-530d3f06523e&pd_rd_w=6TgrH&pd_rd_wg=VzOfs&ref_=sbx_be_s_sparkle_td_asin_0_img',
                    'rev_url':
                    'https://www.amazon.fr/Avantree-Bluetooth-Adaptateur-équipements-Enceintes/product-reviews/B00VWEK4IG/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',
                    'rev_pages': 5 ,
                    'rev_count' : 50
                
                },
                {
                    'prod_id': 2,
                    'prod_url':
                    'https://www.amazon.fr/HP-302-Pack-cartouches-Authentiques/dp/B01LXLFF6H?ref_=Oct_s9_apbd_omwf_hd_bw_bN4CkB&pf_rd_r=V2TTM35Y3ZRBSP9XFVDS&pf_rd_p=3d37f0c5-041f-5707-bec0-94d62f6af77d&pf_rd_s=merchandised-search-10&pf_rd_t=BROWSE&pf_rd_i=340858031&th=1',
                    'rev_url':
                    'https://www.amazon.fr/HP-302-Pack-cartouches-Authentiques/product-reviews/B01LXLFF6H/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',
                    'rev_pages': 10 ,
                    'rev_count' : 100
                
                },
               {
                   'prod_id':3,
                   'prod_url':
                   'https://www.amazon.fr/Logitech-capteur-programmables-ordinateur-portable/dp/B07GS6ZB7T/ref=sr_1_6?brr=1&dchild=1&qid=1618071105&rd=1&s=computers&sr=1-6&th=1',
                   'rev_url':
                   'https://www.amazon.fr/Logitech-capteur-programmables-ordinateur-portable/product-reviews/B07GS6ZB7T/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',
                   'rev_pages': 10 ,
                   'rev_count' : 100
                   
               },
               {
                   'prod_id':4,
                   'prod_url':
                   'https://www.amazon.fr/microSDXC-SanDisk-Adaptateur-Performances-Applicatives/dp/B07FCMKK5X/ref=sr_1_4?brr=1&dchild=1&qid=1618071452&rd=1&s=computers&sr=1-4&th=1',
                   'rev_url':
                   'https://www.amazon.fr/microSDXC-SanDisk-Adaptateur-Performances-Applicatives/product-reviews/B07FCMKK5X/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',
                   'rev_pages': 10 ,
                   'rev_count' : 100
                
                   
               }
            ],
            'cuisine':
            [
              {
                  'prod_id': 5,
                  'prod_url':
                  'https://www.amazon.fr/AmazonBasics-Batterie-Cuisine-Anti-adh%C3%A9sive-pi%C3%A8ces/dp/B07481LPMF/ref=sr_1_5?_encoding=UTF8&c=ts&dchild=1&keywords=Casseroles%2C+po%C3%AAles+et+faitouts&qid=1618067686&s=kitchen&sr=1-5&ts_id=2969505031',
                  'rev_url':
                  'https://www.amazon.fr/AmazonBasics-Batterie-Cuisine-Anti-adh%C3%A9sive-pi%C3%A8ces/product-reviews/B07481LPMF/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews',
                  'rev_pages': 5 ,
                  'rev_count' : 50
              
              },
             {
                'prod_id':6,
                'prod_url':
                'https://www.amazon.fr/céramique-Cadrim-Ensembles-couteaux-Couteaux/dp/B01N5JIZH5/ref=sr_1_4?_encoding=UTF8&c=ts&dchild=1&keywords=Couteaux+et+Ustensiles+de+Cuisine&qid=1618088450&s=kitchen&sr=1-4&ts_id=57698031',
                'rev_url':
                'https://www.amazon.fr/céramique-Cadrim-Ensembles-couteaux-Couteaux/product-reviews/B01N5JIZH5/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',
                'rev_pages': 5 ,
                'rev_count' : 50
                 
                   
              },
             {
                'prod_id':7,
                'prod_url':
                'https://www.amazon.fr/Homgeek-Milk-shake-Bouteilles-Portables-Couvercles/dp/B073XM1TTL?ref_=Oct_s9_apbd_otopr_hd_bw_b3rBLr&pf_rd_r=TC0P3DGNKKBWDWSEDFDM&pf_rd_p=16c350e6-3538-5eed-b91d-8bc2af2c901d&pf_rd_s=merchandised-search-10&pf_rd_t=BROWSE&pf_rd_i=57004031',
                'rev_url':
                'https://www.amazon.fr/Homgeek-Milk-shake-Bouteilles-Portables-Couvercles/product-reviews/B073XM1TTL/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',
                'rev_pages': 5 ,
                'rev_count' : 50 
             },
             {
                'prod_id':8,
                'prod_url':
                'https://www.amazon.fr/Moulinex-Intelligent-Multicuiseur-Recettes-Finition/dp/B00TQILY02?ref_=Oct_s9_apbd_omg_hd_bw_b3rBLr&pf_rd_r=TC0P3DGNKKBWDWSEDFDM&pf_rd_p=8c39304b-cce6-57e6-b16f-e4d3d7c89e6a&pf_rd_s=merchandised-search-10&pf_rd_t=BROWSE&pf_rd_i=57004031',
                'rev_url':
                'https://www.amazon.fr/Moulinex-Intelligent-Multicuiseur-Recettes-Finition/product-reviews/B00TQILY02/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',
                'rev_pages': 5 ,
                'rev_count' : 50  
             }
            ],
             'sports':
             [
              {
                  'prod_id': 9,
                  'prod_url':
                  'https://www.amazon.fr/Gritin-Résistance-Équipement-dExercices-Musculation/dp/B07L9WLKZQ?ref_=Oct_s9_apbd_otopr_hd_bw_bM2FLL&pf_rd_r=6FSMC9T9ZZE6ZNS4JFYA&pf_rd_p=9c37dc66-fb80-563a-854b-d52a81222d81&pf_rd_s=merchandised-search-11&pf_rd_t=BROWSE&pf_rd_i=325615031',
                  'rev_url':
                  'https://www.amazon.fr/Gritin-Résistance-Équipement-dExercices-Musculation/product-reviews/B07L9WLKZQ/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',
                  'rev_pages': 5 ,
                  'rev_count' : 50
              
              },
              {
                  'prod_id': 10,
                  'prod_url':
                  'https://www.amazon.fr/AMONAX-Roller-Roue-Exercice-Abdominaux-Musculation/dp/B081Z7BBMB?ref_=Oct_s9_apbd_oup_hd_bw_bM2FLL&pf_rd_r=6FSMC9T9ZZE6ZNS4JFYA&pf_rd_p=427a8d4e-455a-5ee8-bb93-7b8c5126c94b&pf_rd_s=merchandised-search-11&pf_rd_t=BROWSE&pf_rd_i=325615031',
                  'rev_url':
                  'https://www.amazon.fr/AMONAX-Roller-Roue-Exercice-Abdominaux-Musculation/product-reviews/B081Z7BBMB/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',
                  'rev_pages': 5 ,
                  'rev_count' : 50
                 
              },
              {
                  'prod_id': 11,
                  'prod_url':
                  'https://www.amazon.fr/ISE-Pliable-Motorisé%E2%80%8E-Système-Conduite/dp/B078J7L5WJ/ref=sr_1_5?_encoding=UTF8&c=ts&dchild=1&keywords=Cardio-training&qid=1618089502&s=sports&sr=1-5&ts_id=486046031',
                  'rev_url':
                  'https://www.amazon.fr/ISE-Pliable-Motorisé%E2%80%8E-Système-Conduite/product-reviews/B078J7L5WJ/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',
                  'rev_pages': 5 ,
                  'rev_count' : 50
                 
              },
              {
                  'prod_id': 12,
                  'prod_url':
                  'https://www.amazon.fr/SAGUARO-Chaussures-Minimalistes-Randonnée-Aquatiques/dp/B07N4PW4MZ/ref=sr_1_1_sspa?_encoding=UTF8&c=ts&dchild=1&keywords=Chaussures+de+sport+en+salle&qid=1618089580&s=sports&sr=1-1-spons&ts_id=1766513031&psc=1&spLa=ZW5jcnlwdGVkUXVhbGlmaWVyPUFNSU5OQzZaRjY0WTAmZW5jcnlwdGVkSWQ9QTAwMzczNDMxTDMyUVFYUFE4QkhQJmVuY3J5cHRlZEFkSWQ9QTA3ODYxMDAxWTY1TjFHWlJLNTRZJndpZGdldE5hbWU9c3BfYXRmJmFjdGlvbj1jbGlja1JlZGlyZWN0JmRvTm90TG9nQ2xpY2s9dHJ1ZQ==',
                  'rev_url':
                  'https://www.amazon.fr/SAGUARO-Chaussures-Minimalistes-Randonnée-Aquatiques/product-reviews/B07N4PW4MZ/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews',
                  'rev_pages': 5 ,
                  'rev_count' : 50
                 
              }
             ]
             
}

prod_cols = ['Prod_Title','Prod_Brnd','Prod_Rate','Prod_Eval','Prod_Price','Category','Nb_Revs','Prod_ID']
rev_cols = ['Rev_Title','Rev_Rate','Rev_Bdy','Rev_Hlp','Prod_ID']


# To split settings into another file
settings_file = open("settings.pkl", "wb")
pk.dump(prod_urls, settings_file)
settings_file.close()

settings_file = open("settings.pkl", "rb")
output = pk.load(settings_file)



# End Settings

# Scrapping Product Information

In [11]:

# Scrapping Product Data 
def product_scrapper(driver,prod_category,prod_url):
    driver.get(prod_url)
    prod_soup = BeautifulSoup(driver.page_source,'html.parser')

    # Product Info
    prod_title = prod_soup.find('span',{'id' : 'productTitle' }).text
    prod_brand = prod_soup.find('a',{'id' : 'bylineInfo' }).text
    prod_rate = prod_soup.find('span',{'class' : 'a-icon-alt' }).text
    prod_eval = prod_soup.find('span',{'id' : 'acrCustomerReviewText' }).text
    prod_price = prod_soup.find('span',{'id' : 'priceblock_ourprice' }).text

    result = tuple(map(lambda x : unicodedata.normalize("NFKD",x.strip()),[prod_title,prod_brand,prod_rate,prod_eval,prod_price,prod_category]))
    return result



# Scrapping Reviews Data

In [12]:
# Scrapping Reviews Data
def reviews_scrapper(driver,prod,rev_url,ds):
    driver.get(rev_url)
    rev_soup = BeautifulSoup(driver.page_source,'html.parser')
    reviews_div = rev_soup.find_all('div',{'class':reviews_div_cls})

    for review in reviews_div:
        try :
          review_title = review.find('a',{'data-hook' : 'review-title'}).text
        except AttributeError:
          review_title = 'Null'
        
        try :
          review_rate = review.find('i',{'data-hook' : 'review-star-rating'}).text
        except AttributeError:
          review_rate = '0'
        
        review_body = review.find('span',{'data-hook' : 'review-body'}).text
        
        try :
          review_help = review.find('span',{'data-hook' : 'helpful-vote-statement'}).text
        except AttributeError:
          review_help = '0'
        
        rev_result = tuple(map(lambda x : unicodedata.normalize("NFKD",x.strip()),[review_title,review_rate,review_body,review_help]))
        ds.append(rev_result+(prod,))

    return ds

# Testing data collect

## Setup the web-driver 

In [13]:
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options = options)

## Scrapping Products

In [14]:
lst_prod = [] 
for category,products in prod_urls.items():
    for prod in products:
        product = product_scrapper(driver,category,prod['prod_url'])+(prod['rev_count'],prod['prod_id'])
        lst_prod.append(product)




df_prod = pd.DataFrame(lst_prod)
df_prod.columns = prod_cols
df_prod

,Prod_Title,Prod_Brnd,Prod_Rate,Prod_Eval,Prod_Price,Category,Nb_Revs,Prod_ID
0,Avantree DG40S USB Bluetooth Adaptateur Dongle...,Visiter la boutique Avantree,"4,3 sur 5 étoiles",18 727 évaluations,"12,99 €",informatique,50,1
1,HP 302 Pack de 2 Cartouches d'Encre Noire et T...,Visiter la boutique HP,"4,6 sur 5 étoiles",99 434 évaluations,"31,57 €",informatique,100,2
2,Logitech G502 HERO Souris Gamer Filaire Haute ...,Visiter la boutique Logitech G,"4,7 sur 5 étoiles",37 093 évaluations,"58,46 €",informatique,100,3
3,SanDisk Extreme Carte Mémoire MicroSDXC 128 G...,Visiter la boutique SanDisk,"4,7 sur 5 étoiles",197 427 évaluations,"18,54 €",informatique,100,4
4,Amazon Basics LFFP16027 Batterie de Cuisine An...,Visiter la boutique Amazon Basics,"4,4 sur 5 étoiles",25 914 évaluations,"60,29 €",cuisine,50,5
5,"Couteau en céramique ,Cadrim Ensembles de cou...",Marque : Cadrim,"4,2 sur 5 étoiles",3 353 évaluations,"18,99 €",cuisine,50,6
6,"Homgeek Mini Blender 350W, Blender smoothie et...",Visiter la boutique homgeek,"4,5 sur 5 étoiles",7 469 évaluations,"29,99 €",cuisine,50,7
7,Moulinex Cookeo Multicuiseur Intelligent 100 R...,Visiter la boutique Moulinex,"4,7 sur 5 étoiles",4 397 évaluations,"184,00 €",cuisine,50,8
8,"Gritin Bande Élastique Fitness, [Lot de 5] Ba...",Marque : Gritin,"4,3 sur 5 étoiles",54 623 évaluations,"8,99 €",sports,50,9
9,"AMONAX Roue Abdominale Convertible, AB Roller ...",Visiter la boutique Amonax,"4,6 sur 5 étoiles",7 391 évaluations,"15,99 €",sports,50,10


## Scrapping Reviews Data




In [26]:
# This function help us to generate the dataset of reviews
# and also the dataframe of several products

lst_rev = []
for category,products in prod_urls.items():
    for prod in products:
        for i in range(1,prod['rev_pages']+1):
            lst_rev = reviews_scrapper(driver,prod['prod_id'],f'{prod["rev_url"]}&pageNumber={i}',lst_rev) 

df_rev = pd.DataFrame(lst_rev)
df_rev.columns = rev_cols
df_rev.info()
 
df_rev.to_csv("Review_data.csv")
df_rev = pd.read_csv("Review_data.csv")
df_rev




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750 entries, 0 to 749
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Rev_Title  750 non-null    object
 1   Rev_Rate   750 non-null    object
 2   Rev_Bdy    750 non-null    object
 3   Rev_Hlp    750 non-null    object
 4   Prod_ID    750 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 29.4+ KB


,Unnamed: 0,Rev_Title,Rev_Rate,Rev_Bdy,Rev_Hlp,Prod_ID
0,0,Génial !,"5,0 sur 5 étoiles",J'ai commandé ce produit pour pouvoir écoute...,59 personnes ont trouvé cela utile,1
1,1,En panne après moins de 2 mois d'utilisation,"1,0 sur 5 étoiles","Pour un adaptateur de ce prix, j'aurais espér...",22 personnes ont trouvé cela utile,1
2,2,Compatible pour Dualshock PS4,"5,0 sur 5 étoiles",Acheté pour branchée la manette PS4 sur le P...,38 personnes ont trouvé cela utile,1
3,3,Dongle Bluetooh parfait,"5,0 sur 5 étoiles",J'utilise ce dongle bluetooth sur un PC window...,25 personnes ont trouvé cela utile,1
4,4,Impeccable,"5,0 sur 5 étoiles",Quand je vois certains commentaires de certain...,16 personnes ont trouvé cela utile,1
...,...,...,...,...,...,...
745,745,Plutôt déçu pour un article assez cher.,"2,0 sur 5 étoiles",Produit bien présenté agréable à regarder....,0,12
746,746,Très bon rapport qualité / prix,"5,0 sur 5 étoiles","Très beau produit, arrivé et livré à la da...",Une personne a trouvé cela utile,12
747,747,Chaussures minimalistes,"4,0 sur 5 étoiles",Agréablement surpris. Je marche en ville avec...,0,12
748,748,Excellent rapport pris-prestations.,"4,0 sur 5 étoiles",C'est exact que ces chaussures barefoot offren...,2 personnes ont trouvé cela utile,12


## Clean Data

In [29]:
# deleting string in Rev_Hlp column
def delet_string(s):
   return re.sub("[^0-9]","",s)

# applying the function to clean data  
df_rev['Rev_Hlp']=df_rev.Rev_Hlp.apply(delet_string) 
df_rev['Rev_Rate']=df_rev.Rev_Rate.str.split(' ').apply(lambda x: x[0])
df_rev


,Unnamed: 0,Rev_Title,Rev_Rate,Rev_Bdy,Rev_Hlp,Prod_ID
0,0,Génial !,"5,0",J'ai commandé ce produit pour pouvoir écoute...,59,1
1,1,En panne après moins de 2 mois d'utilisation,"1,0","Pour un adaptateur de ce prix, j'aurais espér...",22,1
2,2,Compatible pour Dualshock PS4,"5,0",Acheté pour branchée la manette PS4 sur le P...,38,1
3,3,Dongle Bluetooh parfait,"5,0",J'utilise ce dongle bluetooth sur un PC window...,25,1
4,4,Impeccable,"5,0",Quand je vois certains commentaires de certain...,16,1
...,...,...,...,...,...,...
745,745,Plutôt déçu pour un article assez cher.,"2,0",Produit bien présenté agréable à regarder....,0,12
746,746,Très bon rapport qualité / prix,"5,0","Très beau produit, arrivé et livré à la da...",,12
747,747,Chaussures minimalistes,"4,0",Agréablement surpris. Je marche en ville avec...,0,12
748,748,Excellent rapport pris-prestations.,"4,0",C'est exact que ces chaussures barefoot offren...,2,12
